## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/underactuated.html?chapter=drake).

In [ ]:
try:
  import pydrake
  import underactuated
except ImportError:
  !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
  from jupyter_setup import setup_underactuated
  setup_underactuated()

# Finding the limit cycle of the Van der Pol oscillator

by setting up a simple trajectory optimization, where the timestep, $h$, is a decision variables.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython import get_ipython

from underactuated.jupyter import AdvanceToAndVisualize, SetupMatplotlibBackend
plt_is_interactive = SetupMatplotlibBackend()

from pydrake.examples.van_der_pol import VanDerPolOscillator
from pydrake.all import DirectCollocation, PiecewisePolynomial, Solve

plant = VanDerPolOscillator()
context = plant.CreateDefaultContext()

dircol = DirectCollocation(plant,
                           context,
                           num_time_samples=61,
                           minimum_timestep=0.01,
                           maximum_timestep=0.5)

# Constrain all timesteps, $h[k]$, to be equal, so the trajectory breaks are evenly distributed.
dircol.AddEqualTimeIntervalsConstraints()

# Initial state on the surface of section (and velocity > .1).
dircol.AddBoundingBoxConstraint([0., 0.1], [0., 10.], dircol.initial_state())

# Periodicity constraint.
# TODO(russt): Replace this with the vectorized version pending drake #8315.
dircol.AddLinearConstraint(dircol.final_state()[0] == dircol.initial_state()[0])
dircol.AddLinearConstraint(dircol.final_state()[1] == dircol.initial_state()[1])

# Help the solver with an initial guess (circular trajectory).
samples = np.linspace(0, 2 * np.pi, 10)
x_guess = np.vstack(
    ([2 * np.sin(t) for t in samples], [2 * np.cos(t) for t in samples]))
initial_x_trajectory = PiecewisePolynomial.FirstOrderHold(samples, x_guess)

dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_trajectory)

fig = plt.figure()
h, = plt.plot([], [], ".-")
plt.xlim((-2.5, 2.5))
plt.ylim((-3., 3.))

def draw_trajectory(t, x):
    h.set_xdata(x[0, :])
    h.set_ydata(x[1, :])
    fig.canvas.draw()
    if plt.get_backend() == u"MacOSX":
        plt.pause(1e-10)

if plt_is_interactive:
    dircol.AddStateTrajectoryCallback(draw_trajectory)

result = Solve(dircol)
assert result.is_success()

x_trajectory = dircol.ReconstructStateTrajectory(result)

x_knots = np.hstack([
    x_trajectory.value(t) for t in np.linspace(x_trajectory.start_time(),
                                               x_trajectory.end_time(), 100)
])
plt.plot(x_knots[0, :], x_knots[1, :]);
